## Drive path

First things first - let's make sure your drive path is set up appropriately. You will want to un-comment only one of these options depending on which operating system you are running. You also may have to tweak the option for your specific set-up (e.g. change the drive letter for Windows).

In [ ]:
### Mac OS X
# drive_path = '/Volumes/Brain2016'

### Windows (a good guess)
# drive_path = 'e:\\'

### Linux (will vary; the following is possibly what Ubuntu will do you will have to change the <username>)
# drive_path = '/media/<username>/Brain2016/'

# Cell Types Database

The Cell Types database contains data derived from single-cell experiments in the mouse visual cortex and lateral geniculate nucleus (LGN). This includes in vitro electrophysiological recordings from cortical slice experiments, morphological reconstructions based on single-neuron fills, and single-cell transcriptomic data from the LGN.

The database can be accessed [on the web](https://celltypes.brain-map.org). This is useful for browsing and familiarizing yourself with the data, but for more in-depth analyses, it is often more efficient to access the data programmatically. The [AllenSDK](https://alleninstitute.github.io/AllenSDK/) provides Python-based tools to support that approach, and this notebook will introduce you to them.

If you have not installed the `allensdk` python module yet, do so from a command prompt:

    > pip install allensdk

## Plotting ephys data from an experiment

Let's start by looking at electrophysiology data from a recorded cell of the mouse visual cortex. The first step is to get access to the Neurodata Without Borders (.nwb) file that contains data from the cell we're interested in. Each recorded cell has its own NWB file.

In [ ]:
import os
import numpy as np
from allensdk.core.cell_types_cache import CellTypesCache

## A CellTypesCache is used to copy and cache data from the cell types database.
## If you create the CellTypesCache instance with no arguments, then data is downloaded
## from the online repository:
# ctc = CellTypesCache()

## Because we have limited internet bandwidth, the data you need is already provided on
## your external hard drive. To ensure this local data is used instead of the online
## repository, create the CellTypesCache instance using the `manifest_file` argument:
ctc = CellTypesCache(manifest_file=os.path.join(drive_path, 'cell_types', 'manifest.json'))

## this saves the NWB file to 'specimen_464212183/ephys.nwb'
data_set = ctc.get_ephys_data(464212183)

<div class="alert alert-info">
<p>
**Documentation:**
[CellTypesCache](http://alleninstitute.github.io/AllenSDK/allensdk.core.html#allensdk.core.cell_types_cache.CellTypesCache),
[CellTypesCache.get_ephys_data()](http://alleninstitute.github.io/AllenSDK/allensdk.core.html#allensdk.core.cell_types_cache.CellTypesCache.get_ephys_data)
</p>
<p>
*Note: you can find these by using the search tool on the main [AllenSDK page](http://alleninstitute.github.io/AllenSDK/)*
</p>
</div>

<div class="alert alert-success">
<p>**EXERCISE**</p>

<p>Use Python code to inspect which data type is returned by `ctc.get_ephys_data()`. (You can verify your answer with the API documentation)</p>

<p>
Try loading the NWB file for the cell with specimen ID of 324493977 and assigning that to a new variable. 
</p>
</div>

The electrophysiological data in a Cell Types NWB file is organized by "sweeps." These sweeps represent specific periods during which a single stimulus is applied to the cell and the response is recorded. Different types of stimuli can be applied to the cell, and similar stimuli are grouped by type on the web app. For example, one-second long step current injections are grouped together under the name "Long Squares." We refer to sweeps by their number, which are integers indicating the order in which the sweeps were collected.

There are typically periods of time during an experiment between sweeps; this may be due to the experimentalist making some on-line adjustments, or may be intentional to allow the cell to return to some baseline after an intense stimulus.

Returning to the example, we now have an object called `data_set` that provides access to the data in the NWB file through built-in methods. We can access sweep-level data like this:

In [ ]:
sweep_number = 30
sweep_data = data_set.get_sweep(sweep_number)

<div class="alert alert-info">
<p>**Documentation:**
[NwbDataSet](http://alleninstitute.github.io/AllenSDK/allensdk.core.html#allensdk.core.nwb_data_set.NwbDataSet), [NwbDataSet.get_sweep()](http://alleninstitute.github.io/AllenSDK/allensdk.core.html#allensdk.core.nwb_data_set.NwbDataSet.get_sweep)
</p>
</div>

<div class="alert alert-success">
<p>**EXERCISE**</p>

<p>`data_set.get_sweep()` returns a dict with four key/value pairs. What are the data types of these values?</p>

</div>

Most of the sweeps in the Cell Types Database are recorded in current-clamp mode, meaning that a current stimulus is injected into the recorded cell, and a voltage response is recorded. The current data is stored in the NWB file in amperes, and the voltage data is in volts. However, it is usually more convenient to work with the data in picoamps and millivolts.

<div class="alert alert-success">
<p>**EXERCISE**</p>

<p>Write code to put the stimulus (current) and response (voltage) from the sweep into variables `i` and `v`, respectively. Convert their units to pA and mV (the raw data are expressed in unscaled A/V units).</p>
</div>

The time points are not included in the NWB file as a waveform, but we can generate one based on the sampling rate:

In [ ]:
sampling_rate = sweep_data["sampling_rate"] # in Hz

<div class="alert alert-success">
<p>**EXERCISE**</p>

<p>Create a new array `t` that is the same length as `v` and `i` and contains the correct time points (in seconds) according to the sampling rate.</p>

<p>See: [numpy.arange()](http://docs.scipy.org/doc/numpy/reference/generated/numpy.arange.html)</p>
</div>

Finally, we can plot the data after importing `matplotlib`.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)
axes[0].plot(t, v, color='black')
axes[1].plot(t, i, color='gray')
axes[0].set(ylabel="mV")
axes[1].set(ylabel="pA", xlabel="seconds")

<div class="alert alert-success">
<p>**EXERCISE**</p>

<p>
Change the x-axis in the above plot to focus on the response during the current step. First do this manually using the toolbar shown below the plots, then do it by modifying the code that generated the plots.
</p>

<p>
See: [matplotlib.axes.set_xlim()](http://matplotlib.org/api/axes_api.html#matplotlib.axes.Axes.set_xlim)
</p>
</div>

Getting the voltage, current stimulus, and time is something we'll want to do a lot. Therefore, we'll make a function that will do it for whatever sweep we give it, and try it out on some new sweeps.

In [ ]:
def get_sweep_v_i_t_from_set(data_set, sweep_number):
    """Given an NwbDataSet and a sweep number, return a tuple containing
    (response, stimulus, time_values).
    """    
    sweep_data = data_set.get_sweep(sweep_number)
    i = sweep_data["stimulus"] # in A
    v = sweep_data["response"] # in V
    i *= 1e12 # to pA
    v *= 1e3 # to mV
    sampling_rate = sweep_data["sampling_rate"] # in Hz
    t = np.arange(len(v)) / sampling_rate
    return v, i, t

In [ ]:
# Retrieve and plot data from two sweeps
sweep_numbers = [38, 40]
fig, axes = plt.subplots(2, 1, sharex=True)

for s in sweep_numbers:
    v, i, t = get_sweep_v_i_t_from_set(data_set, s)
    axes[0].plot(t, v)
    axes[1].plot(t, i, color='gray')

axes[0].set(ylabel="mV")
axes[1].set(ylabel="pA", xlabel="seconds")

<div class="alert alert-success">
<p>**EXERCISE**</p>

<p>
Each NWB file contains multiple sweeps that were recorded from a single neuron. Each sweep has a different type of stimulus — square pulse, ramp, noise, etc.
Print a list of the stimulus type for each sweep in `data_set` (try to print them in order).
</p>

<p>
Hint: The NwbDataSet methods `get_sweep_numbers()` and `get_sweep_metadata()` will be useful. 
</p>

</div>

## Looking at a morphology

As mentioned above, the Cell Types database also contains 3D reconstructions of neuronal morphologies. These data are represented with the [SWC format](http://www.neuronland.org/NLMorphologyConverter/MorphologyFormats/SWC/Spec.html). An SWC file represents the morphology as a connected tree, and each node of the has a location in 3D space as well as a radius.

We'll download a particular cell's reconstruction here.

In [ ]:
# Query the database for all cells that have a morphological reconstruction
cells = ctc.get_cells(require_reconstruction=True)
print "Cells with reconstructions: {:d}".format(len(cells))

In [ ]:
# Download and access an SWC file
cell_id = 464212183
morphology = ctc.get_reconstruction(cell_id) 

<div class="alert alert-info">
**Documentation:** [CellTypesCache.get_reconstruction()](http://alleninstitute.github.io/AllenSDK/allensdk.core.html#allensdk.core.cell_types_cache.CellTypesCache.get_reconstruction), [Morphology](http://alleninstitute.github.io/AllenSDK/allensdk.core.html#allensdk.core.swc.Morphology)
</div>

The AllenSDK contains a module that makes it easier to work with the SWC files. We'll see how the data is represented by looking at the first node.

In [ ]:
morphology = ctc.get_reconstruction(cell_id)
compartment_list = morphology.compartment_list
print compartment_list[0]

Note that the `type` field refers to the type of neuronal compartment:
<table style="align: left">
<tr><th>Compartment type</th><th>Value</th></tr>
<tr><td>Soma</td><td>1</td>
<tr><td>Axon</td><td>2</td>
<tr><td>Dendrite</td><td>3</td>
<tr><td>Apical dendrite</td><td>4</td>
</table>

We can use this data to draw lines between each node and all its children to get a drawing of the cell. We'll do it looking at it from the front (x-y view) and from the side (z-y view).

In [ ]:
fig, axes = plt.subplots(1, 2, sharey=True, sharex=False, figsize=(8, 4))
axes[0].set_aspect('equal')
axes[1].set_aspect('equal')

# Make a line drawing of x-y and y-z views
for c in morphology.compartment_list:
    if c["parent"] == -1: # if this is the root node, don't draw anything
        continue
    p = morphology.compartment_index[c["parent"]]
    axes[0].plot([p['x'], c['x']], [p['y'], c['y']], color='black')
    axes[1].plot([p['z'], c['z']], [p['y'], c['y']], color='black')

axes[0].set_ylabel('y')
axes[0].set_xlabel('x')
axes[1].set_xlabel('z')

<div class="alert alert-success">
<p>**EXERCISE**</p>

<p>
What is the total length of the dendritic processes of this cell?
</p>
<p>
Write a function that takes a specimen ID as a parameter and returns the total length of the dendrites for the cell. Try it on a different cell (the `cells` variable above is a list of cell data entries - each one has an `id` field.).
</div>

## Calculating features of the ephys data

Plotting the data is great, but if we want to do some analysis on things like spike times, spike shapes, etc., we want to go through and extract these features from our raw data. This can be accomplished by using the feature extraction tools in the SDK.

In [ ]:
# First, pick a sweep and plot the membrane potential:
sweep_number = 35
v, i, t = get_sweep_v_i_t_from_set(data_set, sweep_number)

fig = plt.figure()
plt.plot(t, v, color='black')

In [ ]:
# Now let's analyze those spikes:
from allensdk.ephys.ephys_extractor import EphysSweepFeatureExtractor

sweep_ext = EphysSweepFeatureExtractor(t=t, v=v, i=i, start=1.02, end=2.02)
sweep_ext.process_spikes()

print "Avg spike threshold: {:.1f} mV".format(sweep_ext.spike_feature("threshold_v").mean())
print "Avg spike width: {:.2f} ms".format(1e3 * np.nanmean(sweep_ext.spike_feature("width")))

<div class="alert alert-info">
**Documentation:**
[EphysSweepFeatureExtractor](http://alleninstitute.github.io/AllenSDK/allensdk.ephys.html#allensdk.ephys.ephys_extractor.EphysSweepFeatureExtractor)
</div>

The method `spike_feature()` returns a NumPy array of features for each spike. You pass it the name of the feature that you want. Features that can't be calculated for a given spike are set to `NaN`.

We can take a look at all the properties calculated for each spike by the extractor:

In [ ]:
sweep_ext.spike_feature_keys()

We can look at when the spikes occur by looking at the `threshold_t` property (i.e., time of spike threshold).

In [ ]:
spike_times = sweep_ext.spike_feature("threshold_t")

print spike_times[:5]  # print just the first 5 spike times

We can see that the first spikes happen just after the stimulus step begins at 1.02 sec. Let's plot the voltage trace and then put a dot at the time of each spike detected by the extractor.

In [ ]:
fig = plt.figure()
p = plt.plot(t, v, color='black')

min_v = v.min()

v_level = min_v - 5

plt.scatter(spike_times, np.ones(len(spike_times)) * min_v, c='firebrick')
plt.xlim(0.9, 1.2)

We could also get the threshold voltages from the extractor and put dots where the extractor thinks the spikes begin (zooming in a little more to see better):

In [ ]:
fig = plt.figure()
plt.plot(t, v, color='black')

threshold_v = sweep_ext.spike_feature("threshold_v")

# setting zorder puts the dots on top of the trace
plt.scatter(spike_times, threshold_v, s=50, c='firebrick', zorder=20)
plt.xlim(1.015, 1.08)

<div class="alert alert-info">
**Documentation:**
[pyplot.scatter()](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.scatter)
</div>

<div class="alert alert-success">
<p>**EXERCISE**</p>

<p>
Add dots to the above plot to illustrate where the extractor thinks the peaks and fast troughs of the spikes are. Use different colors for these different features.
</p>
</div>

We can use these features to look at things like how particular aspects of firing change throughout the step and at different stimulus levels. Here, we'll look at how the instantaneous firing frequency changes from spike to spike during a step (does it slow down? accelerate? stay the same?).

We'll work through this step-by-step.

<div class="alert alert-success">
<p>**EXERCISE**</p>

<p>
First, calculate the interspike intervals between the spike times shown above and store them in a variable called `isis` (the function [`np.diff()`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.diff.html) may be useful here).
</p>
</div>

Now that we have the interspike intervals, we can invert them to get the instantaneous firing rates.

<div class="alert alert-success">
<p>**EXERCISE**</p>

<p>
Next, invert the interspike intervals to calculate the instantaneous firing rates and store them in the variable `inst_rates`. Then plot that rate vs spike number.
</p>
</div>

Finally, let's plot that for several different sweep amplitudes to see how the response changes.

<div class="alert alert-success">
<p>**EXERCISE**</p>

<p>
Calculate the instantaneous firing frequency for the sweeps numbered 29 through 35 and plot them against spike number on the same graph.
</p>

<p>
On a separate graph, plot the same thing, but normalize each sweep by the instantaneous firing frequency of its first spike.
</p>


</div>

## Pre-computed features in the database

There is also a [set of features](http://help.brain-map.org/download/attachments/8323525/EphysOverview.pdf?version=1&modificationDate=1464378318096) in the Cell Types Database that have already been computed, which could serve as good starting points for further analysis. There are many features computed for each cell, including the cell's input resistance, spike thresholds, spike adaptation, etc.  We can query the database using the SDK to get these features.

In [ ]:
specimen_id = 464212183

# download all features for all cells
features = ctc.get_ephys_features()

# filter down to a specific cell
cell_features = [f for f in features if f['specimen_id'] == specimen_id]

# display the result
cell_features

<div class="alert alert-info">
<p>
**Documentation:**
[CellTypesCache.get_ephys_features()](https://alleninstitute.github.io/AllenSDK/allensdk.core.html?highlight=get_ephys_features#allensdk.core.cell_types_cache.CellTypesCache.get_ephys_features)
</p>
</div>

That's how to get all the ephys features for a given specimen - what if we want a particular feature for all cells?

In [ ]:
tau = np.array([f['tau'] for f in features], dtype=float)
input_resistance = np.array([f['ri'] for f in features], dtype=float)

plt.figure()
plt.scatter(tau, input_resistance, color='seagreen', alpha=0.5)
plt.ylabel("input resistance (MOhm)")
plt.xlabel("membrane time constant (ms)")

Let's use NumPy to fit a regression line to these data and plot it. The function we'll use is [`lstsq()`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.lstsq.html) from the [`numpy.linalg`](http://docs.scipy.org/doc/numpy/reference/routines.linalg.html) module. This function solves the matrix equation 

$A \vec{x} = \vec{y}$

for $\vec{x}$, and it takes $A$ and $\vec{y}$ as its parameters. Here, $\vec{y}$ is a vector of our dependent values (input resistance, in this case). $x$ has the coefficients of the equation we're solving for. Since we're fitting the data to a line of the form:

$y = mx + c$

Our vector of coefficients ($\vec{x}$) will contain $m$ and $c$ like so:

$\vec{x} = \left[ \begin{array}{c}
m \\
c
\end{array} \right]$

So, we need to put together our $A$ matrix so that it makes sense when multiplied with $\vec{x}$ that it would produce $\vec{y}$. To do that, we'll put our independent values (the taus) in the first column of the matrix, and we'll fill the second column with ones. That way, when we multiply $A$ and $\vec{x}$, we'll multiply our tau by $m$ and add $c$.

$A = \left[ \begin{array}{cc}
\tau_0 & 1 \\
\tau_1 & 1 \\
\vdots & \vdots
\end{array} \right]$

To do that, we can use the NumPy [`ones_like()`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.ones_like.html) function to get an array full of ones that's the same size as the one we pass it. We can put two vectors together with the function [`vstack()`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.vstack.html) (since a 1D array is treated as a row), and then use the `T` function to transpose that to turn the rows into columns. Here's the result:

In [ ]:
A = np.vstack([tau, np.ones_like(tau)]).T
A

Now we have the right inputs for `lstsq()`.

<div class="alert alert-success">
<p>**EXERCISE**</p>

<p>
Run the least-squares fit and print out the coefficients `m` and `c` from the fit (note: the function returns a tuple - the coefficients are in the first entry of that tuple).
</p>
</div>

<div class="alert alert-success">
<p>**EXERCISE**</p>

<p>
Repeat the scatter plot of values above, but now plot the fit line on top of the data points.
</p>
</div>

We can look at other pairs of features together.

In [ ]:
updown = np.array([f['upstroke_downstroke_ratio_short_square'] for f in features], dtype=float)
fasttrough = np.array([f['fast_trough_v_long_square'] for f in features], dtype=float)

plt.figure()
plt.scatter(fasttrough, updown, color='seagreen', alpha=0.5)
plt.ylabel("upstroke-downstroke ratio")
plt.xlabel("fast trough depth (mV)")

Here, it looks like there may be roughly two clusters in the data. Maybe they relate to whether the cells are presumably excitatory (spiny) cells or inhibitory (aspiny) cells? Let's query the data to split up the two sets to see.

In [ ]:
cells = ctc.get_cells()

cell_index = { c['id']: c for c in cells}

dendrite_types = ['spiny', 'aspiny']
data = {}

for dendrite_type in dendrite_types:
    type_features = [f for f in features if cell_index[f['specimen_id']]['dendrite_type'] == dendrite_type]
    data[dendrite_type] = {
        "fasttrough": [f['fast_trough_v_long_square'] for f in type_features],
        "updown": [f['upstroke_downstroke_ratio_short_square'] for f in type_features],
    }
    
plt.figure()
for a_type, color in zip(dendrite_types, ["slateblue", "tomato"]):
    plt.scatter(data[a_type]['fasttrough'], data[a_type]['updown'], color=color, label=a_type)
plt.legend(loc='best')
plt.ylabel("upstroke-downstroke ratio")
plt.xlabel("fast trough depth (mV)")

<div class="alert alert-success">
<p>**EXERCISE**</p>

<p>
Pick two other features and plot them against each other. Find a pair you think is most interesting.
</p>
</div>

The database also contains morphological features, including things like the maximum distance from the soma, number of stems emerging from the soma, and others:

In [ ]:
morph_features = ctc.get_morphology_features()

cell_morph_features = [f for f in morph_features if f['specimen_id'] == specimen_id]
print cell_morph_features

## GLIF models

To use the models in the Cell Types database, we'll use another set of tools in the SDK. Here's how to download and run a particular GLIF model that we know the ID of.

In [ ]:
from allensdk.api.queries.glif_api import GlifApi
from allensdk.model.glif.glif_neuron import GlifNeuron

glif_model_id = 472299313 # Level 5 GLIF for neuron with specimen ID = 464212183
glif_api = GlifApi()

glif_api.get_neuronal_model(glif_model_id)
neuron_config = glif_api.get_neuron_config()
neuron = GlifNeuron.from_dict(neuron_config)

In [ ]:
# Run the stimulus from a given sweep
sweep_number = 31
sweep_data = data_set.get_sweep(sweep_number)
i = sweep_data["stimulus"]
sampling_rate = sweep_data["sampling_rate"]
neuron.dt = 1.0 / sampling_rate
index_range = sweep_data["index_range"]

output = neuron.run(i[index_range[0]:index_range[1]])

<div class="alert alert-info">
**Documentation:**
[GlifApi.get_neuronal_model()](http://alleninstitute.github.io/AllenSDK/allensdk.api.queries.html?highlight=get_neuronal_model#allensdk.api.queries.glif_api.GlifApi.get_neuronal_model), [GlifApi.get_neuron_config()](http://alleninstitute.github.io/AllenSDK/allensdk.api.queries.html?highlight=get_neuronal_model#allensdk.api.queries.glif_api.GlifApi.get_neuron_config), [GlifNeuron.from_dict()](http://alleninstitute.github.io/AllenSDK/allensdk.model.glif.html?highlight=glifneuron.from_dict#allensdk.model.glif.glif_neuron.GlifNeuron.from_dict), [GlifNeuron.run()](http://alleninstitute.github.io/AllenSDK/allensdk.model.glif.html?highlight=glifneuron.from_dict#allensdk.model.glif.glif_neuron.GlifNeuron.run)
</div>

We can take a look at the different outputs from the model.

In [ ]:
t = np.arange(0, len(output['voltage'])) * neuron.dt

fig, ax = plt.subplots(1, 1)
ax.plot(t, output['voltage'] * 1e3, label='voltage')
ax.plot(t, output['threshold'] * 1e3, label='threshold')
ax.legend(loc="best")

spike_times = output['interpolated_spike_times']
ax.scatter(spike_times, np.ones(len(spike_times)) * 30.0, c='k')
ax.set(ylabel="mV", xlabel="seconds", xlim=(0, 2))

Several versions of GLIF models exist for many neurons, with additional factors that increase their complexity (and, ideally, improve their reproduction of the original data). We can compare these fits directly - in this case, the level-5 version we've been looking at (that includes factors like reset rules, after-spike currents, and threshold adaption) with a simpler level-2 version (that only includes reset rules). 

<div class="alert alert-success">
<p>**EXERCISE**</p>

<p>
Look at the Level-2 GLIF model for the same cell (GLIF model ID of 472299317). In what ways does it differ from the Level-5 version? Does that change for different sweeps?</p>
</div>

## Biophysical models

**NOTE: You must have NEURON installed and configured to work with Python for this section to work correctly**

All biophysical models are implemented using the [NEURON](http://www.neuron.yale.edu/neuron) simulation environment. Documentation for the [NEURON API can be found here](http://www.neuron.yale.edu/neuron/static/new_doc/index.html). A biophysical model can be downloaded and run in a similar way as the GLIF models using the AllenSDK.

In [ ]:
from allensdk.api.queries.biophysical_api import BiophysicalApi

bp_api = BiophysicalApi()
bp_api.cache_stimulus = False # don't download the stimulus waveforms (NWB file)
neuronal_model_id = 473863035

bp_api.cache_data(neuronal_model_id, working_directory='neuronal_model')

<div class="alert alert-info">
**Documentation:**
[BiophysicalAPI](http://alleninstitute.github.io/AllenSDK/allensdk.api.queries.html?highlight=biophysicalapi#allensdk.api.queries.biophysical_api.BiophysicalApi)
</div>

We need to compile the .mod files that describe the biophysical mechanisms before using the model.

**OSX/Linux users:** Run the cell below to compile .mod files.

**Windows users:** 

* Start -> All Programs -> NEURON 7.4 -> mknrndll
* Browse to the dynamicbrain2016\celltypes\neuronal_model\modfiles directory
* Click `Make nrnmech.dll`

In [ ]:
import sys
if sys.platform == 'win32':
    print "WARNING: modfiles must be compiled manually in windows; see above."
else:
    # Execute the NEURON program `nrnivmodl`
    os.system('nrnivmodl neuronal_model/modfiles')

Now that that's done, let's see what else is in the folder.

In [ ]:
print(os.listdir('neuronal_model/'))

Now that we have our files sorted, we'll configure the model.

In [ ]:
# How to get NEURON to import on a Windows systems (not necessary for OS X or Linux)
import os, sys
sys.path.append("C:\\nrn\\lib\\python")
os.putenv('PATH', os.getenv('PATH') + ";C:\\nrn\\bin;C:\\nrn\\mingw\\bin")

In [ ]:
from allensdk.model.biophys_sim.config import Config
from allensdk.model.biophysical.utils import Utils

cwd = os.getcwd()
os.chdir(os.path.join(cwd, 'neuronal_model'))

description = Config().load("manifest.json")
utils = Utils(description) # object for setting up the model

manifest = description.manifest
morphology_path = manifest.get_path("MORPHOLOGY")
utils.generate_morphology(morphology_path)
utils.load_cell_parameters()

# Now let's set up the stimulus
h = utils.h # get access to the NEURON environment through "h"
stim = h.IClamp(h.soma[0](0.5)) # insert a current clamp stimulus in the middle of the soma
stim.amp = 0.2 # nA
stim.dur = 1000 # ms
stim.delay = 1000 # ms

# set up variable time step for faster execution
h.cvode_active(1)
h.cvode.atolscale("cai", 1e-4)
h.cvode.maxstep(10)

vec = utils.record_values() # save the voltage and time values

h.tstop = 3000 # run it for 3 sec

os.chdir(os.path.join(cwd))

<div class="alert alert-info">
**Documentation:** [biophys_sim.config.Config](http://alleninstitute.github.io/AllenSDK/allensdk.model.biophys_sim.html?highlight=biophys_sim.config#allensdk.model.biophys_sim.config.Config), [biophysical.utils](http://alleninstitute.github.io/AllenSDK/allensdk.model.biophysical.html#module-allensdk.model.biophysical.utils), [nrn.IClamp](http://www.neuron.yale.edu/neuron/static/docs/help/neuron/neuron/mech.html#IClamp), [nrn.Cvode](http://www.neuron.yale.edu/neuron/static/new_doc/simctrl/cvode.html)

Now let's run the model and see what we get.

In [ ]:
h.finitialize()
h.run()

plt.figure()
plt.plot(vec["t"], vec["v"], color='black')

We had to go down into the `neuronal_model` directory to compile and run the biophysical model, but let's go back up for the next part.

In [ ]:
os.chdir("..") # ".." refers to the parent directory on a UNIX-like system

# Homework

### Data access

<div class="alert alert-success">
<p>**HOMEWORK**</p>

<p>
Load data (voltage, current, time) from a long-square sweep. Try to use NumPy methods to figure out the amplitude of the current step applied during this sweep. Can you also figure out when the step begins? Would your method work with any step amplitude?
</p>
</div>

<div class="alert alert-success">
<p>**HOMEWORK**</p>

<p>
Load sweeps for three different stimulus protocols (ramp, short square, and long square). Look for the appropriate sweep numbers using the web app.
</p>

<p>
Plot both the voltage traces and the current stimuli that elicited them. Try to adjust the plot axis limits to view the most relevant part of the data.
</p>

<p>
**Hint:** The URL for a particular specimen ID has the form <code>http://celltypes.brain-map.org/mouse/experiment/electrophysiology/000000000</code> where <code>000000000</code> is the specimen ID.
</p>
</div>

### Morphological data

<div class="alert alert-success">
<p>**HOMEWORK**</p>

<p>
Plot the morphology for the cell with a specimen ID of 473943881. Color the axonal and dendritic compartments differently.
</p>
</div>

<div class="alert alert-success">
<p>**HOMEWORK**</p>

<p>
Use the function you wrote earlier to calculate the total length of the dendritic compartments of every cell with a morphological reconstruction. Plot a histogram of those values.
</p>
</div>

### Feature analysis

<div class="alert alert-success">
<p>**HOMEWORK**</p>

<p>
Plot the average upstroke/downstroke ratio (on short squares) of all the cells, split up by Cre line (note: use the key `transgenic_line` to access that information). Add error bars to show the standard deviation.
</p>
</div>

<div class="alert alert-success">
<p>**HOMEWORK**</p>

<p>
Use the feature extractor to plot the waveform of every spike in a given sweep on top of each other. How much (or little) variability there is in the spike waveform during the course of a sweep? Is there a consistent trend from the first spike to the last?
</p>
</div>

<div class="alert alert-success">
<p>**HOMEWORK**</p>

<p>
The membrane time constant ($\tau$) and input resistance ($R_m$) can be used together to estimate the total capacitance of the cell ($C_m$), since $\tau = R_m C_m$. Estimate the capacitance of each cell that has a reconstruction. Then, plot the capacitance against the total dendritic length you calculated above to see if there is any relationship between the two.
</p>
</div>

<div class="alert alert-success">
<p>**BONUS HOMEWORK**</p>

<p>
Train an LDA classifier to distinguish between spiny/aspiny based on the upstroke/downstroke ratio and fast trough depth.</p>
</div>

### Single-cell models

<div class="alert alert-success">
<p>**HOMEWORK**</p>
Load the GLIF model with the ID 472424802. Use the code below to generate a sine-wave modulated input with random noise. Stimulate the model with that input.
</p>
<p>
Try a few instances of that input (with different random noise) to see how that affects the time of the spikes.
</p>
</div>

In [ ]:
sampling_rate = 20000 # in Hz
dt = 1. / sampling_rate
t = np.arange(0, sampling_rate * 2) * dt
cycles_per_second = 2
intensity = 200
i = intensity * np.sin(t * (2 * np.pi * cycles_per_second)) + np.random.normal(0, 7 * intensity, len(t))
i *= 1e-12 # from pA to A

<div class="alert alert-success">
<p>**BONUS HOMEWORK (if you have NEURON set up)**</p>
<p>
Load the biophysically-detailed model shown in the *Biophysical models* section above. Use the feature extractor to analyze spike features and plot them as in the *Calculating features of the ephys data* section. 
</p>
</div>